## Load ERA5 data


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import requests
import h5py
import sys; sys.path.append('../')
import openai

from backend.geoloc import geocode, get_closest_pixel

In [ ]:
data_dir = Path('../data/era5/')

# Open h5py file
f = h5py.File(data_dir / 'sample.h5', 'r')

In [ ]:
# Load u wind, v wind, and temperature
u = f['u10'][0]
v = f['v10'][0]
temp = f['t2m'][0]

In [ ]:
# Image represents the Earth: get closest pixel to target location
data = geocode(city='San Francisco', state='CA', country_code='US')
lat_idx, lon_idx = get_closest_pixel(data['lat'], data['lon'])

# Get the data
u_sf = u[lat_idx, lon_idx]
v_sf = v[lat_idx, lon_idx]
temp_sf = temp[lat_idx, lon_idx]
print('Wind speed: {:.2f} [m/s]'.format(np.sqrt(u_sf**2 + v_sf**2)))
print("Temperature: {:.2f} [C]".format(temp_sf - 273.15)) # kelvin to celsius


In [ ]:
# Plot on a map San Francisco
plt.figure(figsize=(10, 10))
plt.imshow(temp, cmap='coolwarm')
plt.scatter(lon_idx, lat_idx, c='b', s=100, marker='*')
plt.title('San Francisco')

### Time Series

In [ ]:
data_dir = Path('../data/era5/')

# Open h5py file
f = h5py.File(data_dir / 'single_week.h5', 'r')

# Load u wind, v wind, and temperature
u = f['u10'][:, lat_idx, lon_idx]
v = f['v10'][:, lat_idx, lon_idx]
temp = f['t2m'][:, lat_idx, lon_idx]

In [ ]:
# set font latex
plt.rc('font', family='serif')
# 16 font size
plt.rcParams.update({'font.size': 18})

fig, axs = plt.subplots(2,1, figsize=(12, 6))
axs[0].scatter(np.arange(28), np.sqrt(u**2 + v**2), c='b', s=100, marker='2')
axs[0].set_title('Wind Speed [m/s]')
axs[0].set_xticks([0, 4, 8, 12, 16, 20, 24, 28])
axs[0].set_xticklabels(['0', '1', '2', '3', '4', '5', '6', '7'])

axs[1].scatter(np.arange(28), temp - 273.15, c='r', s=100, marker='2')
axs[1].set_title('Temperature [C]')
axs[1].set_xticks([0, 4, 8, 12, 16, 20, 24, 28])
axs[1].set_xticklabels(['0', '1', '2', '3', '4', '5', '6', '7'])

plt.tight_layout()

## OpenAI GPT-3

In [ ]:
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]
target_variables = ['u10', 'v10', 't2m', 'tp', 'r850', 'sp']

# build dictionary of the above with their description and units in square brackets
# U wind component = eastward wind 
# V wind component = northward wind
variables = {'u10': 'Eastward wind  [km/h]',
            'v10': 'Northward wind [km/h]',
            't2m': 'Surface temperature [°C]',
            'tp': 'Total precipitation [mm]',
            'r850': 'Relative humidity [%]',
            'sp': 'Surface pressure [hPa]'}

# Convert from m/s to km/h, and kelvin to celsius
conversion_factors = {  'u10': {'factor': 3.6, 'offset': 0},
                        'v10': {'factor': 3.6, 'offset': 0},
                        't2m': {'factor': 1, 'offset': -273.15},
                        'tp': {'factor': 1, 'offset': 0},
                        'r850': {'factor': 1, 'offset': 0},
                        'sp': {'factor': 0.01, 'offset': 0}}

def rescale(x, var):
    return x * conversion_factors[var]['factor'] + conversion_factors[var]['offset']


prompt = ""

time_idx = 0

for i, var in enumerate(target_variables):
    prompt += "{} is {:.1f}\n".format(variables[var], rescale(f[var][time_idx, lat_idx, lon_idx], var))


print(prompt)

In [ ]:
final_prompt = prompt + "\n Given the above information, what is the weather like? How much is the temperatur in Fahreneit?"


response = openai.Completion.create(
    model="text-davinci-002",
    prompt=final_prompt,
    temperature=0,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=[" Human:", " AI:"] #\n
)

In [ ]:
print(response['choices'][0]['text'][2:])